In [33]:
import pandas as pd
import re

from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [38]:
CLINICAL_DATA_PATH_1 = "../data/broad_tcga/clinical/gdac.broadinstitute.org_STAD.Clinical_Pick_Tier1.Level_4.2016012800.0.0.tar.gz"

METH_PATH = "../data/broad_tcga/methylation/gdac.broadinstitute.org_STAD.Merge_methylation__humanmethylation450__jhu_usc_edu__Level_3__within_bioassay_data_set_function__data.Level_3.2016012800.0.0.tar.gz"
MIRNA_PATH_HISEQ = "../data/broad_tcga/miRSeq/gdac.broadinstitute.org_STAD.Merge_mirnaseq__illuminahiseq_mirnaseq__bcgsc_ca__Level_3__miR_gene_expression__data.Level_3.2016012800.0.0.tar.gz"

RNA_PATH_HISEQ_v1 = '../data/broad_tcga/mRNASeq/gdac.broadinstitute.org_STAD.Merge_rnaseq__illuminahiseq_rnaseq__bcgsc_ca__Level_3__gene_expression__data.Level_3.2016012800.0.0.tar.gz'
RNA_PATH_GA_v1 = '../data/broad_tcga/mRNASeq/gdac.broadinstitute.org_STAD.Merge_rnaseq__illuminaga_rnaseq__bcgsc_ca__Level_3__gene_expression__data.Level_3.2016012800.0.0.tar.gz'
RNA_PATH_HISEQ_v2 = ''
RNA_PATH_HISEQ_normalized_v2 = '../data/broad_tcga/mRNASeq/gdac.broadinstitute.org_STAD.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.Level_3.2016012800.0.0.tar.gz'

In [3]:
clin = pd.read_csv(CLINICAL_DATA_PATH_1, index_col=0, sep='\t')
rna  = pd.read_csv(RNA_PATH_HISEQ_normalized_v2, compression='gzip', index_col=0, header=0, sep='\t').iloc[1:, :]

/home/dicer/miniconda3/envs/ortholog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,2

In [4]:
rna.head()

,TCGA-3M-AB46-01A-11R-A414-31,TCGA-3M-AB47-01A-22R-A414-31,TCGA-B7-5816-01A-21R-1602-13,TCGA-B7-5818-01A-11R-1602-13,TCGA-B7-A5TI-01A-11R-A31P-31,TCGA-B7-A5TJ-01A-11R-A31P-31,TCGA-B7-A5TK-01A-12R-A36D-31,TCGA-B7-A5TN-01A-21R-A31P-31,TCGA-BR-4187-01A-01R-1131-13,TCGA-BR-4191-01A-02R-1131-13,...,TCGA-VQ-AA6A-01A-11R-A414-31,TCGA-VQ-AA6B-01A-11R-A414-31,TCGA-VQ-AA6D-01A-11R-A414-31,TCGA-VQ-AA6F-01A-31R-A414-31,TCGA-VQ-AA6G-01A-11R-A414-31,TCGA-VQ-AA6I-01A-11R-A414-31,TCGA-VQ-AA6J-01A-11R-A414-31,TCGA-VQ-AA6K-01A-11R-A414-31,TCGA-ZA-A8F6-01A-23R-A36D-31,TCGA-ZQ-A9CR-01A-11R-A39E-31
././@LongLink,,,,,,,,,,,,,,,,,,,,,
?|100130426,0.0000,0.0000,0.0000,0.0000,0.3799,0.0000,0.0000,0.6151,0.0000,0.0000,...,0.0000,0.3189,0.0000,0.0000,0.0000,0.0000,0.2466,0.4134,0.0000,0.5668
?|100133144,11.3383,14.2000,3.6576,4.7020,11.5125,23.9514,5.1939,18.7530,3.2408,21.6635,...,26.1744,28.1937,32.4153,18.6091,33.7625,35.0189,27.6770,28.6855,29.3939,19.8016
?|100134869,17.1023,7.8408,12.1906,41.1395,17.1613,19.7869,0.6053,18.4578,1.6204,19.0769,...,14.3662,16.5156,21.5482,17.2753,20.2653,30.1605,19.2494,21.1378,15.4703,8.5379
?|10357,40.5136,17.9592,0.4171,6.5488,21.1362,65.8505,36.5857,36.2882,42.5116,82.1251,...,11.5541,10.0447,15.9546,21.9117,8.4115,21.6463,35.6665,11.5371,22.0386,29.3701
?|10431,2163.3818,880.4082,750.2867,748.5265,1346.5950,769.3458,1307.3577,958.8683,1020.4578,1127.4678,...,357.7864,780.6259,903.2378,1033.0264,733.7431,519.9928,709.6427,702.4735,561.9835,1874.0178


In [50]:
mirna = pd.read_csv(MIRNA_PATH_HISEQ, index_col=0, header=0, sep='\t')

/home/dicer/miniconda3/envs/ortholog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,2,4,5,7,8,10,11,13,14,16,17,19,20,22,23,25,26,28,29,31,32,34,35,37,38,40,41,43,44,46,47,49,50,52,53,55,56,58,59,61,62,64,65,67,68,70,71,73,74,76,77,79,80,82,83,85,86,88,89,91,92,94,95,97,98,100,101,103,104,106,107,109,110,112,113,115,116,118,119,121,122,124,125,127,128,130,131,133,134,136,137,139,140,142,143,145,146,148,149,151,152,154,155,157,158,160,161,163,164,166,167,169,170,172,173,175,176,178,179,181,182,184,185,187,188,190,191,193,194,196,197,199,200,202,203,205,206,208,209,211,212,214,215,217,218,220,221,223,224,226,227,229,230,232,233,235,236,238,239,241,242,244,245,247,248,250,251,253,254,256,257,259,260,262,263,265,266,268,269,271,272,274,275,277,278,280,281,283,284,286,287,289,290,292,293,295,296,298,299,301,302,304,305,307,308,310,311,313,314,316,317,319,320,322,323,325,326,328,329,331,332,334,335,337,338,340,341,343,344,346,347,349,350,352,353,3

In [52]:
mirna.head()

,././@LongLink,TCGA-3M-AB46-01A-11R-A415-13,TCGA-3M-AB46-01A-11R-A415-13.1,TCGA-3M-AB46-01A-11R-A415-13.2,TCGA-3M-AB47-01A-22R-A415-13,TCGA-3M-AB47-01A-22R-A415-13.1,TCGA-3M-AB47-01A-22R-A415-13.2,TCGA-B7-5816-01A-21R-1602-13,TCGA-B7-5816-01A-21R-1602-13.1,TCGA-B7-5816-01A-21R-1602-13.2,...,TCGA-VQ-AA6J-01A-11R-A415-13.2,TCGA-VQ-AA6K-01A-11R-A415-13,TCGA-VQ-AA6K-01A-11R-A415-13.1,TCGA-VQ-AA6K-01A-11R-A415-13.2,TCGA-ZA-A8F6-01A-23R-A36E-13,TCGA-ZA-A8F6-01A-23R-A36E-13.1,TCGA-ZA-A8F6-01A-23R-A36E-13.2,TCGA-ZQ-A9CR-01A-11R-A39F-13,TCGA-ZQ-A9CR-01A-11R-A39F-13.1,TCGA-ZQ-A9CR-01A-11R-A39F-13.2
0,miRNA_ID,read_count,reads_per_million_miRNA_mapped,cross-mapped,read_count,reads_per_million_miRNA_mapped,cross-mapped,read_count,reads_per_million_miRNA_mapped,cross-mapped,...,cross-mapped,read_count,reads_per_million_miRNA_mapped,cross-mapped,read_count,reads_per_million_miRNA_mapped,cross-mapped,read_count,reads_per_million_miRNA_mapped,cross-mapped
1,hsa-let-7a-1,26683,8209.125811,N,29589,6691.767107,N,29869,6567.375126,N,...,N,39139,7933.978852,N,54671,7432.004587,N,41193,9709.891276,N
2,hsa-let-7a-2,53150,16351.798406,N,58553,13242.185928,N,59792,13146.623372,Y,...,N,77645,15739.640460,N,109007,14818.469098,N,81668,19250.537729,Y
3,hsa-let-7a-3,26645,8197.434967,N,29662,6708.276587,N,30410,6686.326210,N,...,N,39001,7906.004477,N,54614,7424.255977,N,40889,9638.233301,N
4,hsa-let-7b,77373,23804.095918,N,35444,8015.917852,N,89528,19684.755440,N,...,N,53523,10849.800713,N,45293,6157.154319,N,54589,12867.556498,N


In [34]:
def get_src_mask(df: pd.DataFrame, ids: list):
    '''
    Gets the boolean mask for samples corresponding to ids.
    
    :param df - dataframe in expected format (rows are omics' features, columns are tcga samples)
    :param ids - list of tcga ids of interest. E.g. ids of patients with SRC.
    '''
    rgx = r"|".join(ids)
    src_mask = list(map(lambda x: bool(re.search(rgx, x)),
                        map(lambda x: "-".join(x.lower().split("-")[:3]), df.columns)))
    return src_mask


# Perform Mann-Whitney U-test - nonparametric test for testing different expression for signet ring cells
# An equivalent to ANOVA test is Kruskal-Wallis one way analysis, which was employed by broad gdac,
# as a nonparametric, robust equivalent to one-way anova.
# Let's have a function that takes a dataframe and computes this test for histologic type, in 2 versions:
# kruskal-wallis and mann-whitney for src (with direction).

def associations_with_src(df: pd.DataFrame, src_ids: list) -> dict:
    '''
    Computes dict of associations (test statistic and p-value) 
     of features in df with sample appartenance in src_ids.
    
    :param df
    :param src_ids
    '''
    
    src_mask    = get_src_mask(rna, src_ids)
    nonsrc_mask = [not i for i in src_mask]
    
    res = {}
    for idx, row in df.iterrows():
        src_vals    = row[src_mask]
        nonsrc_vals = row[nonsrc_mask]
        try:
            mwu = mannwhitneyu(nonsrc_vals, src_vals, alternative='two-sided')
        except ValueError:
            # assume all nrs are 0 and equal
            res[idx] = [0, 1]
            
        res[idx] = [mwu[0], mwu[1]]
    
    return res


def compute_src_statistics(df: pd.DataFrame, src_ids: list) -> pd.DataFrame:
    '''
    Computes statistics for associating molecular features of SRC subtype.
    
    :param df - samples are tcga ids of all patients in background
    :param src_ids - list of ids of samples corresponding to patients with SRC tumors.
    '''
    
    r = associations_with_src(df, src_ids)
    r = pd.DataFrame.from_dict(r, orient='columns').transpose()
    r.columns = ['test-statistic', 'p-value']
    r.sort_values(['p-value'], ascending=True, inplace=True)
    
    # Add multiple test correction
    mtcorr = multipletests(r['p-value'],
                          alpha=0.05,
                          method='fdr_bh')
    r['p-value_corrected'] = mtcorr[1]
    r['p-value_reject']    = mtcorr[0]
    
    
    return r


def compute_allomics_src_statistics(paths: dict, src_ids: list) -> pd.DataFrame:
    '''
    Computes statistics of association with SRC subtype for different multiomics data.
    
    :param paths - dict of omics_type -> Path()
    :param src_ids - list of ids of src type
    
    :return df with columns [feature, omics_type, test-statistic, p-value, p-value-corrected, p-value-reject]
    '''
    res = {}
    for omics_type, path in paths.items():
        
        # Every file will need separate processing
        # Have a loading function for each type of omics data
#         df = pd.read_csv(path, sep='\t', compression='gzip')
        
        res[omics_type] = compute_src_statistics(df, src_ids)

In [35]:
paths = {
    "methylation": METH_PATH,
    "mirseq"     : MIRNA_PATH_HISEQ,
    "mrnaseq"    : RNA_PATH_HISEQ_normalized_v2
}

src_ids = ['tcga-fp-a8cx', 'tcga-kb-a93g', 'tcga-vq-a94u', 'tcga-br-a453',
           'tcga-fp-a4bf', 'tcga-hj-7597', 'tcga-vq-a8dz', 'tcga-vq-a8pc',
           'tcga-vq-a8pm', 'tcga-vq-a8po', 'tcga-vq-a8pz', 'tcga-vq-a94p',
           'tcga-vq-aa6k']

# res = compute_src_statistics(rna, src_ids)
res = compute_allomics_src_statistics(paths, src_ids)

In [37]:
res[res['p-value_reject']==True]

,test-statistic,p-value,p-value_reject,p-value_corrected
CABP2|51475,1823.0,5.858640e-09,True,0.000120
OR11H6|122748,1241.0,1.580589e-07,True,0.001623


In [75]:
# mannwhitneyu([0,5,0],[51,50,50],alternative='two-sided')

MannwhitneyuResult(statistic=0.0, pvalue=0.07219819770165753)